#1- Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
import csv 
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
from google.colab import files

# !pip install --upgrade openpyxl

#2- Fetching Required Features from Ergast Developer API

Race Data

In [ ]:
#Webscrapping race data from Ergast website
Race = {'season': [],'round': [],'circuit_id': [], 'country': [], 'date': []}

for year in list(range(2012,2022)):
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            Race['season'].append(int(item['season']))
        except:
            Race['season'].append(None)

        try:
            Race['round'].append(int(item['round']))
        except:
            Race['round'].append(None)

        try:
            Race['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            Race['circuit_id'].append(None)
            
        try:
            Race['country'].append(item['Circuit']['Location']['country'])
        except:
            Race['country'].append(None)

        try:
            Race['date'].append(item['date'])
        except:
            Race['date'].append(None)

Race = pd.DataFrame(Race)
Race

,season,round,circuit_id,country,date
0,2012,1,albert_park,Australia,2012-03-18
1,2012,2,sepang,Malaysia,2012-03-25
2,2012,3,shanghai,China,2012-04-15
3,2012,4,bahrain,Bahrain,2012-04-22
4,2012,5,catalunya,Spain,2012-05-13
...,...,...,...,...,...
194,2021,18,rodriguez,Mexico,2021-11-07
195,2021,19,interlagos,Brazil,2021-11-14
196,2021,20,losail,Qatar,2021-11-21
197,2021,21,jeddah,Saudi Arabia,2021-12-05


Result Data

In [ ]:
laps = []
for year in np.array(Race.season.unique()):
    laps.append([year, list(Race[Race.season == year]['round'])])
  
Result = {'season': [],'round':[],'circuit_id':[], 'driver': [], 'date_of_birth': [],'nationality': [],'constructor': [], 'points': [], 'podium': []}

for n in list(range(len(laps))):
    for i in laps[n][1]:
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(laps[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:
                Result['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except:
                Result['season'].append(None)

            try:
                Result['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except:
                Result['round'].append(None)

            try:
                Result['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except:
                Result['circuit_id'].append(None)

            try:
                Result['driver'].append(item['Driver']['driverId'])
            except:
                Result['driver'].append(None)
            
            try:
                Result['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except:
                Result['date_of_birth'].append(None)
                
            try:
                Result['nationality'].append(item['Driver']['nationality'])
            except:
                Result['nationality'].append(None)

            try:
                Result['constructor'].append(item['Constructor']['constructorId'])
            except:
                Result['constructor'].append(None)

            try:
                Result['points'].append(int(item['points']))
            except:
                Result['points'].append(None)

            try:
                Result['podium'].append(int(item['position']))
            except:
                Result['podium'].append(None)

Result = pd.DataFrame(Result)
Result

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,points,podium
0,2012,1,albert_park,button,1980-01-19,British,mclaren,25.0,1
1,2012,1,albert_park,vettel,1987-07-03,German,red_bull,18.0,2
2,2012,1,albert_park,hamilton,1985-01-07,British,mclaren,15.0,3
3,2012,1,albert_park,webber,1976-08-27,Australian,red_bull,12.0,4
4,2012,1,albert_park,alonso,1981-07-29,Spanish,ferrari,10.0,5
...,...,...,...,...,...,...,...,...,...
4160,2021,22,yas_marina,latifi,1995-06-29,Canadian,williams,0.0,16
4161,2021,22,yas_marina,giovinazzi,1993-12-14,Italian,alfa,0.0,17
4162,2021,22,yas_marina,russell,1998-02-15,British,williams,0.0,18
4163,2021,22,yas_marina,raikkonen,1979-10-17,Finnish,alfa,0.0,19


Driver Statistics Data

In [ ]:
Driver_stats = {'season': [], 'round':[], 'driver': [], 'driver_points': [], 'driver_wins': [], 'driver_position': []}

for n in list(range(len(laps))):
    for i in laps[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(laps[n][0], 1))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                Driver_stats['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                Driver_stats['season'].append(None)

            try:
                Driver_stats['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                Driver_stats['round'].append(None)
                                         
            try:
                Driver_stats['driver'].append(item['Driver']['driverId'])
            except:
                Driver_stats['driver'].append(None)
            
            try:
                Driver_stats['driver_points'].append(int(item['points']))
            except:
                Driver_stats['driver_points'].append(None)
            
            try:
                Driver_stats['driver_wins'].append(int(item['wins']))
            except:
                Driver_stats['driver_wins'].append(None)
                
            try:
                Driver_stats['driver_position'].append(int(item['position']))
            except:
                Driver_stats['driver_position'].append(None)
            
Driver_stats = pd.DataFrame(Driver_stats)
Driver_stats

,season,round,driver,driver_points,driver_wins,driver_position
0,2012,1,button,25,1,1
1,2012,1,vettel,18,0,2
2,2012,1,hamilton,15,0,3
3,2012,1,webber,12,0,4
4,2012,1,alonso,10,0,5
...,...,...,...,...,...,...
4135,2021,1,mick_schumacher,0,0,16
4136,2021,1,gasly,0,0,17
4137,2021,1,latifi,0,0,18
4138,2021,1,alonso,0,0,19


Constructor Statistics Data

In [ ]:
laps = laps[1:]

constructor_stats = {'season': [],'round':[], 'constructor': [], 'constructor_points': [], 'constructor_wins': [], 'constructor_position': []}

for n in list(range(len(laps))):
    for i in laps[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(laps[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:
                constructor_stats['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                constructor_stats['season'].append(None)

            try:
                constructor_stats['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                constructor_stats['round'].append(None)
                                         
            try:
                constructor_stats['constructor'].append(item['Constructor']['constructorId'])
            except:
                constructor_stats['constructor'].append(None)
            
            try:
                constructor_stats['constructor_points'].append(int(item['points']))
            except:
                constructor_stats['constructor_points'].append(None)
            
            try:
                constructor_stats['constructor_wins'].append(int(item['wins']))
            except:
                constructor_stats['constructor_wins'].append(None)
                
            try:
                constructor_stats['constructor_position'].append(int(item['position']))
            except:
                constructor_stats['constructor_position'].append(None)

constructor_stats = pd.DataFrame(constructor_stats)
constructor_stats

,season,round,constructor,constructor_points,constructor_wins,constructor_position
0,2013,1,ferrari,30.0,0,1
1,2013,1,lotus_f1,26.0,1,2
2,2013,1,red_bull,23.0,0,3
3,2013,1,mercedes,10.0,0,4
4,2013,1,force_india,10.0,0,5
...,...,...,...,...,...,...
1843,2021,22,alphatauri,142.0,0,6
1844,2021,22,aston_martin,77.0,0,7
1845,2021,22,williams,23.0,0,8
1846,2021,22,alfa,13.0,0,9


#3- Merging all acquired data

In [ ]:
df1 = pd.merge(Race, Result, how='inner',on=['season', 'round', 'circuit_id']).drop(['points'], axis = 1)                                                  
df2 = pd.merge(df1, Driver_stats, how='left',on=['season', 'round', 'driver'])
F1Data = pd.merge(df2, constructor_stats, how='left',on=['season', 'round', 'constructor'])
F1Data

,season,round,circuit_id,country,date,driver,date_of_birth,nationality,constructor,podium,driver_points,driver_wins,driver_position,constructor_points,constructor_wins,constructor_position
0,2012,1,albert_park,Australia,2012-03-18,button,1980-01-19,British,mclaren,1,25.0,1.0,1.0,NaN,NaN,NaN
1,2012,1,albert_park,Australia,2012-03-18,button,1980-01-19,British,mclaren,1,25.0,1.0,1.0,NaN,NaN,NaN
2,2012,1,albert_park,Australia,2012-03-18,button,1980-01-19,British,mclaren,1,25.0,1.0,1.0,NaN,NaN,NaN
3,2012,1,albert_park,Australia,2012-03-18,button,1980-01-19,British,mclaren,1,25.0,1.0,1.0,NaN,NaN,NaN
4,2012,1,albert_park,Australia,2012-03-18,button,1980-01-19,British,mclaren,1,25.0,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8092,2021,22,yas_marina,UAE,2021-12-12,latifi,1995-06-29,Canadian,williams,16,NaN,NaN,NaN,23.0,0.0,8.0
8093,2021,22,yas_marina,UAE,2021-12-12,giovinazzi,1993-12-14,Italian,alfa,17,NaN,NaN,NaN,13.0,0.0,9.0
8094,2021,22,yas_marina,UAE,2021-12-12,russell,1998-02-15,British,williams,18,NaN,NaN,NaN,23.0,0.0,8.0
8095,2021,22,yas_marina,UAE,2021-12-12,raikkonen,1979-10-17,Finnish,alfa,19,NaN,NaN,NaN,13.0,0.0,9.0


#4- Data Cleaning and Pre-processing

Converting age of drivers to datetime format

In [ ]:
F1Data['date'] = pd.to_datetime(F1Data.date)
F1Data['date_of_birth'] = pd.to_datetime(F1Data.date_of_birth)
F1Data['driver_age'] = F1Data.apply(lambda x:relativedelta(x['date'], x['date_of_birth']).years, axis=1)
F1Data.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

Handling null values

In [ ]:
for col in ['driver_points', 'driver_wins', 'driver_position', 'constructor_points','constructor_wins' , 'constructor_position']:
    F1Data[col].fillna(0, inplace = True)
    F1Data[col] = F1Data[col].map(lambda x: int(x))
F1Data.dropna(inplace = True )

#5- Saving and Exporting Cleaned File

In [ ]:
F1Data.to_csv('F1Data.csv')
files.download('F1Data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>